<a href="https://colab.research.google.com/github/manya0601/TECHDES/blob/main/waste_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os, shutil, random

# ✅ Set your main dataset path on Google Drive
base_dir = '/content/drive/MyDrive/Final-pakkaaa!!!'
source_dir = os.path.join(base_dir, 'dataset')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

def split_data(source_dir, train_dir, val_dir, test_dir, split_ratio=(0.7, 0.15, 0.15)):
    classes = os.listdir(source_dir)
    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        if not os.path.isdir(cls_path):
            continue  # Skip files that are not folders
        files = os.listdir(cls_path)
        random.shuffle(files)
        n = len(files)
        train_end = int(split_ratio[0]*n)
        val_end = train_end + int(split_ratio[1]*n)
        for i, f in enumerate(files):
            src_file = os.path.join(cls_path, f)
            if i < train_end:
                dst = os.path.join(train_dir, cls, f)
            elif i < val_end:
                dst = os.path.join(val_dir, cls, f)
            else:
                dst = os.path.join(test_dir, cls, f)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy(src_file, dst)

# 🧠 Run the split
split_data(source_dir, train_dir, val_dir, test_dir)
print("✅ Dataset split into train, val, and test folders successfully!")


✅ Dataset split into train, val, and test folders successfully!


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # ✅ 6 classes = 6 neurons

])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 82944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    10,616,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,637,126 (40.58 MB)

 Trainable params: 10,637,126 (40.58 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Use the paths defined in the previous cell
train_gen = train_datagen.flow_from_directory(train_dir, target_size=(150,150), batch_size=32, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_dir, target_size=(150,150), batch_size=32, class_mode='categorical')

history = model.fit(train_gen, validation_data=val_gen, epochs=10)

Found 1766 images belonging to 6 classes.
Found 377 images belonging to 6 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.2643 - loss: 2.4001 - val_accuracy: 0.3846 - val_loss: 1.4580
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.4334 - loss: 1.4276 - val_accuracy: 0.5252 - val_loss: 1.2558
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.5116 - loss: 1.2824 - val_accuracy: 0.5570 - val_loss: 1.1622
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.5722 - loss: 1.1025 - val_accuracy: 0.5889 - val_loss: 1.1196
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.6260 - loss: 0.9767 - val_accuracy: 0.5544 - val_loss: 1.1961
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.6812 - loss: 0.8618 - val_accuracy: 0.6127 - val_loss: 1.0845
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.7463 - loss: 0.7030 - val_accuracy: 0.6180 - val_loss: 1.1105
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.8047 - loss: 0.5838 - val_accuracy: 0.6127 - val_loss

In [ ]:
loss, acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {acc*100:.2f}%")


12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 509ms/step - accuracy: 0.5922 - loss: 1.4477
Validation Accuracy: 58.62%


In [ ]:
model.save('waste_classifier.h5')
print("✅ Model saved as waste_classifier.h5")


✅ Model saved as waste_classifier.h5


In [ ]:
!cp waste_classifier.h5 /content/drive/MyDrive/


In [ ]:
# Function to record short webcam video
def record_video(filename='live_video.mp4', duration=5):
    js = Javascript('''
      async function recordVideo() {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = "🎥 Start Recording";
        div.appendChild(capture);
        document.body.appendChild(div);
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        const recorder = new MediaRecorder(stream);
        const chunks = [];
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.onstop = e => {
          const completeBlob = new Blob(chunks, { type: 'video/mp4' });
          const reader = new FileReader();
          reader.onload = () => {
            google.colab.kernel.invokeFunction('notebook.SaveVideo', [reader.result], {});
          };
          reader.readAsDataURL(completeBlob);
        };
        recorder.start();
        await new Promise(r => capture.onclick = r);
        recorder.stop();
      }
      recordVideo();
    ''')
    display(js)

def save_video(data_url):
    header, encoded = data_url.split(",", 1)
    data = b64decode(encoded)
    with open('live_video.mp4', 'wb') as f:
        f.write(data)

output.register_callback('notebook.SaveVideo', save_video)
